API docs for the WikiSRATMicroService
===

**SRAT code is all in SQL, run from this notebook.**  
This code is within the GitHub repository. The result of this will be a table of NHDplus catchments. 

**Instances where load reduction is greater than the load created by an NHDplus catchment:**
- After weekly check in call, decided that it can be negative, so leave as raw value
- Otherwise can set threshold to where can't reduce loads beyond a proportion of the current load

In [1]:
import requests
import numpy as np
import pandas as pd
import geopandas as gpd 
from requests.auth import HTTPBasicAuth
import json
import os
import psycopg2
from sqlalchemy import create_engine 
from pathlib import Path
import pyarrow
from StringParser import StringParser
from DatabaseAdapter import DatabaseAdapter
from DatabaseFormatter import DatabaseFormatter
from DatabaseMakeTable import DatabaseMakeTable

# Run WikiSRATMicroService API

## Open Connection to WikiSRATMicroService Database

To fetch information to assist in formating requests to the WikiSRATMicroService API.

In [2]:
# GET THE DATABASE CONFIG INFORMATION USING A CONFIG FILE. 
# THE FILE IS IN THE GITIGNORE SO WILL REQUIRE BEING SENT VIA EMAIL.

config_file = json.load(open('db_config.json'))
PG_CONFIG = config_file['PG_CONFIG']

_host = PG_CONFIG['host'],
_database = PG_CONFIG['database'],
_user = PG_CONFIG['user'],
_password = PG_CONFIG['password'],
_port = PG_CONFIG['port']

### Create Connection to database with `psycopg2`, to read and write

In [3]:
# Create connection to database using `psycopg2`

_PG_Connection = psycopg2.connect(
        host=PG_CONFIG['host'],
        database=PG_CONFIG['database'],
        user=PG_CONFIG['user'],
        password=PG_CONFIG['password'],
        port=PG_CONFIG['port'])

### Create Connection using SQLAlchemy, to read with GeoPandas
An alternate way to connect, using sqlalchemy, written by Sarah.  
Benefits include:
- Doesn't require knowledge of column names
- Correctly auto-parses datatypes
- Auto-converts column named 'geom' from a string into a geometry datatype compatible with  GeoPandas and most plotting libraries.

In [4]:
%%time
# Connect to database with sqlalchemy

db_connection_url = "postgresql://{}:{}@{}:{}/{}".format(_user[0], _password[0], _host[0], _port, _database[0])
con = create_engine(db_connection_url)  

CPU times: user 16.1 ms, sys: 3.93 ms, total: 20 ms
Wall time: 24.2 ms


## Fetch data for formating requests to the WikiSRATMicroService API


In [5]:
%%time

# GET THE MODELED LOADS FROM THE DRWI DATABASE, DERIVED FROM MMW MODEL RUNS

_PG_Connection.set_isolation_level(0)
_cur = _PG_Connection.cursor()
_cur.execute("select * from databmpapi.drb_loads_raw order by huc12;")  
# _cur.execute("select * from databmpapi.drb_loads_raw where huc12 in ('020402030902', '020402030901');")  

_dbdata = _cur.fetchall()
print(len(_dbdata))

484
CPU times: user 17.2 ms, sys: 8.61 ms, total: 25.8 ms
Wall time: 612 ms


In [6]:
_dbdata?

Type:        list
String form: [(1, '020401010101', Decimal('83.25'), Decimal('131587.70'), Decimal('144086.00'), Decimal('11916 <...> 00421087362510914, 0.0325741241610815, 0.124997522620959, 0.149069147756057, 0.0349152439841768)]
Length:      484
Docstring:  
Built-in mutable sequence.

If no argument is given, the constructor creates a new empty list.
The argument must be an iterable if specified.


## Run WikiSRATMicroService API


In [7]:
# CREATE A COUPLE HELPER FUNCTIONS TO RUN THE MICROSERVICE
def respond(err, res=None):
    return {
        'statusCode': '400' if err else '200',
        'body': err.args[0] if err else json.dumps(res),
        'headers': {
            'Content-Type': 'application/json',
            'Access-Control-Allow-Origin': '*'
        },
    }

def lambda_handler(event, context):
    try:
        data = StringParser.parse(event['body'])
        db = DatabaseAdapter(_database[0], _user[0], _host[0], _port, _password[0], _flag)
        input_array = DatabaseAdapter.python_to_array(data)
        return respond(None, db.run_model(input_array))
    except AttributeError as e:
        return respond(e)

In [11]:
# Prepare the input payload body for the MicroService request
_body = DatabaseFormatter.parse(_dbdata)
# _body = '[{"huc12": "020402010101", "tpload_hp": 10, "tpload_crop": 10, "tpload_wooded": 10, "tpload_open": 10, "tpload_barren": 10, "tpload_ldm": 10, "tpload_mdm": 10, "tpload_hdm": 10, "tpload_wetland": 10, "tpload_farman": 10, "tpload_tiledrain": 10, "tpload_streambank": 10, "tpload_subsurface": 10, "tpload_pointsource": 10, "tpload_septics": 10, "tnload_hp": 10, "tnload_crop": 10, "tnload_wooded": 10, "tnload_open": 10, "tnload_barren": 10, "tnload_ldm": 10, "tnload_mdm": 10, "tnload_hdm": 10, "tnload_wetland": 10, "tnload_farman": 10, "tnload_tiledrain": 10, "tnload_streambank": 10, "tnload_subsurface": 10, "tnload_pointsource": 10, "tnload_septics": 10, "tssload_hp": 10, "tssload_crop": 10, "tssload_wooded": 10, "tssload_open": 10, "tssload_barren": 10, "tssload_ldm": 10, "tssload_mdm": 10, "tssload_hdm": 10, "tssload_wetland": 10, "tssload_tiledrain": 10, "tssload_streambank": 10}, {"huc12": "020402010102", "tpload_hp": 10, "tpload_crop": 10, "tpload_wooded": 10, "tpload_open": 10, "tpload_barren": 10, "tpload_ldm": 10, "tpload_mdm": 10, "tpload_hdm": 10, "tpload_wetland": 10, "tpload_farman": 10, "tpload_tiledrain": 10, "tpload_streambank": 10, "tpload_subsurface": 10, "tpload_pointsource": 10, "tpload_septics": 10, "tnload_hp": 10, "tnload_crop": 10, "tnload_wooded": 10, "tnload_open": 10, "tnload_barren": 10, "tnload_ldm": 10, "tnload_mdm": 10, "tnload_hdm": 10, "tnload_wetland": 10, "tnload_farman": 10, "tnload_tiledrain": 10, "tnload_streambank": 10, "tnload_subsurface": 10, "tnload_pointsource": 10, "tnload_septics": 10, "tssload_hp": 10, "tssload_crop": 10, "tssload_wooded": 10, "tssload_open": 10, "tssload_barren": 10, "tssload_ldm": 10, "tssload_mdm": 10, "tssload_hdm": 10, "tssload_wetland": 10, "tssload_tiledrain": 10, "tssload_streambank": 10}, {"huc12": "020402010103", "tpload_hp": 10, "tpload_crop": 10, "tpload_wooded": 10, "tpload_open": 10, "tpload_barren": 10, "tpload_ldm": 10, "tpload_mdm": 10, "tpload_hdm": 10, "tpload_wetland": 10, "tpload_farman": 10, "tpload_tiledrain": 10, "tpload_streambank": 10, "tpload_subsurface": 10, "tpload_pointsource": 10, "tpload_septics": 10, "tnload_hp": 10, "tnload_crop": 10, "tnload_wooded": 10, "tnload_open": 10, "tnload_barren": 10, "tnload_ldm": 10, "tnload_mdm": 10, "tnload_hdm": 10, "tnload_wetland": 10, "tnload_farman": 10, "tnload_tiledrain": 10, "tnload_streambank": 10, "tnload_subsurface": 10, "tnload_pointsource": 10, "tnload_septics": 10, "tssload_hp": 10, "tssload_crop": 10, "tssload_wooded": 10, "tssload_open": 10, "tssload_barren": 10, "tssload_ldm": 10, "tssload_mdm": 10, "tssload_hdm": 10, "tssload_wetland": 10, "tssload_tiledrain": 10, "tssload_streambank": 10}]'

In [9]:
_body?

Type:        str
String form: [{"huc12": "020401010101", "tpload_hp": 607.6, "tpload_crop": 362.6, "tpload_wooded": 36.6, "tplo <...> hdm": 2799.2, "tssload_wetland": 65.3, "tssload_tiledrain": 0.0, "tssload_streambank": 780437.0}]
Length:      483251
Docstring:  
str(object='') -> str
str(bytes_or_buffer[, encoding[, errors]]) -> str

Create a new string object from the given object. If encoding or
errors is specified, then the object must expose a data buffer
that will be decoded using the given encoding and error handler.
Otherwise, returns the result of object.__str__() (if defined)
or repr(object).
encoding defaults to sys.getdefaultencoding().
errors defaults to 'strict'.


### for Base Model

In [10]:
%%time

# FOR ALL DRWI HUC12s, FEED THROUGH THE MICROSERVICE TO GET SUB-BASIN ATTENUATION
# The database adapter routine flag can either be 'base' or 'restoration', depending on if you want these
# projects to be removed from the attenuation routine. Restoration projects come from what was enetered in
# through FieldDoc.

_flag = 'base'

# RUN THE HUC12s THROUGH THE MICROSERVICE
_r = dict(lambda_handler({"body": _body},None))

CPU times: user 355 ms, sys: 38.4 ms, total: 393 ms
Wall time: 6.95 s


In [9]:
# Explore the API response
_r?

Type:        dict
String form: {'statusCode': '200', 'body': '{"huc12s": {"020401010101": {"huc12": "020401010101", "tpload_hp": <...> 15247}}}}}', 'headers': {'Content-Type': 'application/json', 'Access-Control-Allow-Origin': '*'}}
Length:      3
Docstring:  
dict() -> new empty dictionary
dict(mapping) -> new dictionary initialized from a mapping object's
    (key, value) pairs
dict(iterable) -> new dictionary initialized as if via:
    d = {}
    for k, v in iterable:
        d[k] = v
dict(**kwargs) -> new dictionary initialized with the name=value pairs
    in the keyword argument list.  For example:  dict(one=1, two=2)


In [10]:
# Extract the NHD Loads from the response
_nhdloads = dict(json.loads(_r['body']))['huc12s']
_nhdloads?

Type:        dict
String form: {'020401010101': {'huc12': '020401010101', 'tpload_hp': 602.07665754054, 'tpload_crop': 360.72733 <...> 0436867554431356, 'tssloadrate_total': 25055.0654476434, 'tssloadrate_conc': 41.5994419315247}}}}
Length:      484
Docstring:  
dict() -> new empty dictionary
dict(mapping) -> new dictionary initialized from a mapping object's
    (key, value) pairs
dict(iterable) -> new dictionary initialized as if via:
    d = {}
    for k, v in iterable:
        d[k] = v
dict(**kwargs) -> new dictionary initialized with the name=value pairs
    in the keyword argument list.  For example:  dict(one=1, two=2)


In [11]:
# Explore selection of data for a HUC12
print(dict(json.loads(_r['body']))['huc12s']['020402010101']['catchments'])

{'4481881': {'comid': 4481881, 'tploadrate_total': 8.67678906666698, 'tploadate_conc': 0.00480867068316327, 'tnloadrate_total': 205.478173246534, 'tnloadrate_conc': 0.188308808608071, 'tssloadrate_total': 12430.9823731758, 'tssloadrate_conc': 10.6329585165398}, '4481681': {'comid': 4481681, 'tploadrate_total': 45.9269178229512, 'tploadate_conc': 0.0139356826495012, 'tnloadrate_total': 1098.12798966065, 'tnloadrate_conc': 0.333206840298741, 'tssloadrate_total': 128512.512354698, 'tssloadrate_conc': 38.9947698116636}, '4481279': {'comid': 4481279, 'tploadrate_total': 9.3369485692889, 'tploadate_conc': 0.0870941956538323, 'tnloadrate_total': 260.499630512306, 'tnloadrate_conc': 2.34191601922763, 'tssloadrate_total': 4135.72100063273, 'tssloadrate_conc': 49.6417992751781}, '4481935': {'comid': 4481935, 'tploadrate_total': 47.8207522003034, 'tploadate_conc': 0.107891339771179, 'tnloadrate_total': 1122.82652578667, 'tnloadrate_conc': 2.53327797292518, 'tssloadrate_total': 29233.9090668937, '

# Load Results into Database (SKIP if DONE)

### If you have done this for the latest data, skip this block.

Loading results into the database is a time consuming process (~20-30 minutes), so it only needs to be run once for every data update. 

In [134]:
# GET THE TOTAL NUMBER OF ROWS TO PRINT THE % COMPLETED LATER ON

t = 0
for huc12s, huc12 in _nhdloads.items():
    for comid in _nhdloads[huc12s]['catchments']:
        t += 1
        
t

19496

## Save "Base" Model Results (i.e. no BMPs) to Database

In [135]:
# LOAD THE RESULTS INTO A DATABASE FOR REVIEW, CONSULT MSC94@DREXEL.EDU FOR MORE INFORMATION (MAY REQUIRE PERMISSION)
# CREATE THE TABLE TO CACHE THE API OUTPUT
# This uses an imported function to create the table. This is necessary to get the COMID geometries

# SET THE TABLE NAME AND CREATE TABLE
tablename_base = 'base_run'
new = DatabaseMakeTable(_database[0], _user[0], _host[0], _port, _password[0], tablename_base)
new.make_table()


Table Created


In [136]:
%%time

# LOADING RESULTS INTO THE DB CAN TAKE 10-15 MINUTES with a fast internet connection 
# (~ 10 min for Mike; ~14 min for Anthony at work),
# and it times-out after only 20% complete with a slower connection (i.e. Anthony's home)

c = 0
prog_update = 0.1
print('0%', end='--->')
for huc12s, huc12 in _nhdloads.items():
    for comid in _nhdloads[huc12s]['catchments']:
        update_arr = [int(_nhdloads[huc12s]['catchments'][comid]['comid']),
                      _nhdloads[huc12s]['catchments'][comid]['tploadrate_total'],
                      _nhdloads[huc12s]['catchments'][comid]['tploadate_conc'],
                      _nhdloads[huc12s]['catchments'][comid]['tnloadrate_total'],
                      _nhdloads[huc12s]['catchments'][comid]['tnloadrate_conc'],
                      _nhdloads[huc12s]['catchments'][comid]['tssloadrate_total'],
                      _nhdloads[huc12s]['catchments'][comid]['tssloadrate_conc']]
        update_arr = [x if x != None else -9999 for x in update_arr]
        _PG_Connection.set_isolation_level(0)
        _cur = _PG_Connection.cursor()
        _cur.execute("insert into wikiwtershedoutputs.{} values ({},{},{},{},{},{},{})"
                     ";".format(tablename_base, update_arr[0],update_arr[1],update_arr[2],update_arr[3],update_arr[4],update_arr[5],update_arr[6]))
        c += 1
        if c == int(t * prog_update - 1):
            print('{}%'.format(int(prog_update*100)), end='--->')
            prog_update = round(prog_update+0.1,1)
print('done')

0%--->10%--->20%--->30%--->40%--->50%--->60%--->70%--->80%--->90%--->100%--->done
Wall time: 7min 46s


## Repeat for Restoration Results

Reruns the WikiSRATMicroService API with restoration BMPs and saves response to the database.

In [12]:
# FOR ALL DRWI HUC12s, FEED THROUGH THE MICROSERVICE TO GET SUB-BASIN ATTENUATION

# NOW RUN THE ATTENUATION WITH THE BMPs ADDED IN

_flag = 'restoration'

_r = dict(lambda_handler({"body": _body},None))
print('done')

done


In [138]:
# GET THE TOTAL NUMBER OF ROWS TO PRINT THE % COMPLETED LATER ON
_nhdloads = dict(json.loads(_r['body']))['huc12s']
t = 0
for huc12s, huc12 in _nhdloads.items():
    for comid in _nhdloads[huc12s]['catchments']:
        t += 1

In [139]:
# LOAD THE RESULTS INTO A DATABASE FOR REVIEW, CONSULT MSC94@DREXEL.EDU FOR MORE INFORMATION (MAY REQUIRE PERMISSION)
# CREATE THE TABLE TO CACHE THE API OUTPUT
tablename_rest = 'restoration_run'
new = DatabaseMakeTable(_database[0], _user[0], _host[0], _port, _password[0], tablename_rest)
new.make_table()

Table Created


In [140]:
%%time

# LOADING RESULTS INTO THE DB CAN TAKE 10-15 MINUTES with a fast internet connection 
# (~ 10 min for Mike; ~14 min for Anthony at work),
# and it times-out after only 20% complete with a slower connection (i.e. Anthony's home)

c = 0
prog_update = 0.1
print('0%', end='--->')
for huc12s, huc12 in _nhdloads.items():
    for comid in _nhdloads[huc12s]['catchments']:
        update_arr = [int(_nhdloads[huc12s]['catchments'][comid]['comid']),
                      _nhdloads[huc12s]['catchments'][comid]['tploadrate_total'],
                      _nhdloads[huc12s]['catchments'][comid]['tploadate_conc'],
                      _nhdloads[huc12s]['catchments'][comid]['tnloadrate_total'],
                      _nhdloads[huc12s]['catchments'][comid]['tnloadrate_conc'],
                      _nhdloads[huc12s]['catchments'][comid]['tssloadrate_total'],
                      _nhdloads[huc12s]['catchments'][comid]['tssloadrate_conc']]
        update_arr = [x if x != None else -9999 for x in update_arr]
        _PG_Connection.set_isolation_level(0)
        _cur = _PG_Connection.cursor()
        _cur.execute("insert into wikiwtershedoutputs.{} values ({},{},{},{},{},{},{})"
                     ";".format(tablename_rest, update_arr[0],update_arr[1],update_arr[2],update_arr[3],update_arr[4],update_arr[5],update_arr[6]))
        c += 1
        if c == int(t * prog_update - 1):
            print('{}%'.format(int(prog_update*100)), end='--->')
            prog_update = round(prog_update+0.1,1)

print('done')

0%--->10%--->20%--->30%--->40%--->50%--->60%--->70%--->80%--->90%--->100%--->done
Wall time: 8min 7s


# ABOVE CODE (Load Results) ONLY NEEDS TO BE RUN ONCE

Use labs to get outline to collapse code to be run once. Further edits to the data frames for API output can be done here.

# Read Results in Pandas and Save to Local Storage

In [15]:
# PUT HERE ANY TABLE NAMES CREATED ABOVE, IN CASE THE KERNEL WAS RESTARTED AND YOU DON'T WANT TO RUN THE ABOVE AGAIN
tablename_rest = 'restoration_run'
tablename_base = 'base_run'

## Read from Database into Pandas with `psycopg2`
Using a custom function written by Mike, using the [pyscopg2](https://www.psycopg.org) library.

In [13]:
# Re-connect to database using `psycopg2`(necessary if timed out)

_PG_Connection = psycopg2.connect(
        host=PG_CONFIG['host'],
        database=PG_CONFIG['database'],
        user=PG_CONFIG['user'],
        password=PG_CONFIG['password'],
        port=PG_CONFIG['port'])

In [14]:
%%time

# LOAD THE DATABASE TABLES INTO PANDAS
# geom = NHDplus streamline
# catchment = NHDplus catchment

def postgresql_to_dataframe(conn, select_query, column_names):
    """
    Tranform a SELECT query into a pandas dataframe
    """
    _cur = conn.cursor()
    try:
        _cur.execute(select_query)
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        _cur.close()
        return 1
    
    # We get a list of tupples
    tupples = _cur.fetchall()
    _cur.close()
    
    # Turn it into a pandas dataframe
    df = pd.DataFrame(tupples, columns=column_names)
    return df

colnames = ('comid','tploadrate_total','tploadate_conc','tnloadrate_total','tnloadate_conc','tssloadrate_total',
            'tssloadate_conc','catchment_hectares','watershed_hectares','tploadrate_total_ws','tnloadrate_total_ws',
            'tssloadrate_total_ws','maflowv','geom','geom_catchment', 'cluster', 'fa_name', 'sub_focusarea',
            'nord','nordstop', 'huc12')

cluster_names = ('Brandywine and Christina','Kirkwood - Cohansey Aquifer','Middle Schuylkill','New Jersey Highlands',
                 'Poconos and Kittatinny','Schuylkill Highlands','Upper Lehigh','Upstream Suburban Philadelphia')


base_model_select = 'SELECT * FROM wikiwtershedoutputs.{}'.format(tablename_base)
rest_model_select = 'SELECT * FROM wikiwtershedoutputs.{}'.format(tablename_rest)

base_df = postgresql_to_dataframe(_PG_Connection, base_model_select, colnames)
rest_df = postgresql_to_dataframe(_PG_Connection, rest_model_select, colnames)

print('done')

done
CPU times: user 859 ms, sys: 1.79 s, total: 2.64 s
Wall time: 3min 18s


In [15]:
# Note that most columns are read as string objects, and ...
# geometry datatypes are not compatbile with GeoPandas and common Python plotting functions.
base_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19496 entries, 0 to 19495
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   comid                 19496 non-null  int64  
 1   tploadrate_total      19496 non-null  object 
 2   tploadate_conc        19496 non-null  object 
 3   tnloadrate_total      19496 non-null  object 
 4   tnloadate_conc        19496 non-null  object 
 5   tssloadrate_total     19496 non-null  object 
 6   tssloadate_conc       19496 non-null  object 
 7   catchment_hectares    19496 non-null  object 
 8   watershed_hectares    19496 non-null  object 
 9   tploadrate_total_ws   19496 non-null  object 
 10  tnloadrate_total_ws   19496 non-null  object 
 11  tssloadrate_total_ws  19496 non-null  object 
 12  maflowv               19496 non-null  object 
 13  geom                  19266 non-null  object 
 14  geom_catchment        19496 non-null  object 
 15  cluster            

In [16]:
base_df.head(3)

,comid,tploadrate_total,tploadate_conc,tnloadrate_total,tnloadate_conc,tssloadrate_total,tssloadate_conc,catchment_hectares,watershed_hectares,tploadrate_total_ws,...,tssloadrate_total_ws,maflowv,geom,geom_catchment,cluster,fa_name,sub_focusarea,nord,nordstop,huc12
0,2612952,0.6351,0.0200,11.5479,0.2536,1173.0077,8.8315,6.3851,981.0000,0.1216,...,53.7069,6.6760,01050000E06A7F00000100000001020000C007000000B3...,01060000206A7F0000010000000103000000010000001B...,drb,None,NaN,74295.0,74299.0,020401010101
1,2612782,51.2284,0.0188,758.0229,0.2414,22417.1069,7.4794,524.1138,906.3900,0.1147,...,45.6521,6.1910,01050000E06A7F00000100000001020000C015000000D8...,01060000206A7F000001000000010300000001000000DF...,drb,None,NaN,74297.0,74299.0,020401010101
2,2612920,797.8818,0.0373,8339.1431,0.4056,366215.4767,17.1041,3191.3462,4174.8300,0.2170,...,99.5046,27.1790,01050000E06A7F00000100000001020000C063000000E1...,01060000206A7F000001000000010300000001000000C1...,drb,None,NaN,74294.0,74299.0,020401010101


## Read from Database using SQLAlchemy and GeoPandas
An alternate way to connect, using sqlalchemy, written by Sarah.  
Benefits include:
- Doesn't require knowledge of column names
- Correctly auto-parses datatypes
- Auto-converts column named 'geom' from a string into a geometry datatype compatible with  GeoPandas and most plotting libraries.

In [52]:
%%time
# Connect to database with sqlalchemy
from sqlalchemy import create_engine  

db_connection_url = "postgresql://{}:{}@{}:{}/{}".format(_user[0], _password[0], _host[0], _port, _database[0])
con = create_engine(db_connection_url)  

CPU times: user 327 µs, sys: 0 ns, total: 327 µs
Wall time: 331 µs


In [53]:
%%time
# import data with geopandas compatible geometry
import geopandas as gpd 

base_model_select = 'SELECT * FROM wikiwtershedoutputs.{}'.format(tablename_base)
rest_model_select = 'SELECT * FROM wikiwtershedoutputs.{}'.format(tablename_rest)

base_df = gpd.read_postgis(base_model_select, con)
rest_df = gpd.read_postgis(rest_model_select, con)

CPU times: user 2.47 s, sys: 1.03 s, total: 3.5 s
Wall time: 42.3 s


In [54]:
base_df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 19496 entries, 0 to 19495
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   comid                 19496 non-null  int64   
 1   tploadrate_total      19496 non-null  float64 
 2   tploadate_conc        19496 non-null  float64 
 3   tnloadrate_total      19496 non-null  float64 
 4   tnloadate_conc        19496 non-null  float64 
 5   tssloadrate_total     19496 non-null  float64 
 6   tssloadate_conc       19496 non-null  float64 
 7   catchment_hectares    19496 non-null  float64 
 8   watershed_hectares    19496 non-null  float64 
 9   tploadrate_total_ws   19496 non-null  float64 
 10  tnloadrate_total_ws   19496 non-null  float64 
 11  tssloadrate_total_ws  19496 non-null  float64 
 12  maflowv               19496 non-null  float64 
 13  geom                  19266 non-null  geometry
 14  geom_catchment        19496 non-null  object  

In [55]:
rest_df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 19496 entries, 0 to 19495
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   comid                 19496 non-null  int64   
 1   tploadrate_total      19496 non-null  float64 
 2   tploadate_conc        19496 non-null  float64 
 3   tnloadrate_total      19496 non-null  float64 
 4   tnloadate_conc        19496 non-null  float64 
 5   tssloadrate_total     19496 non-null  float64 
 6   tssloadate_conc       19496 non-null  float64 
 7   catchment_hectares    19496 non-null  float64 
 8   watershed_hectares    19496 non-null  float64 
 9   tploadrate_total_ws   19496 non-null  float64 
 10  tnloadrate_total_ws   19496 non-null  float64 
 11  tssloadrate_total_ws  19496 non-null  float64 
 12  maflowv               19496 non-null  float64 
 13  geom                  19266 non-null  geometry
 14  geom_catchment        19496 non-null  object  

In [56]:
base_df.head(3)

,comid,tploadrate_total,tploadate_conc,tnloadrate_total,tnloadate_conc,tssloadrate_total,tssloadate_conc,catchment_hectares,watershed_hectares,tploadrate_total_ws,...,tssloadrate_total_ws,maflowv,geom,geom_catchment,cluster,fa_name,sub_focusarea,nord,nordstop,huc12
0,2612952,0.6351,0.0200,11.5479,0.2536,1173.0077,8.8315,6.3851,981.00,0.1216,...,53.7069,6.676,MULTILINESTRING Z ((531509.616 4697096.509 0.0...,01060000206A7F0000010000000103000000010000001B...,drb,None,NaN,74295.0,74299.0,020401010101
1,2612782,51.2284,0.0188,758.0229,0.2414,22417.1069,7.4794,524.1138,906.39,0.1147,...,45.6521,6.191,MULTILINESTRING Z ((531592.400 4699047.630 0.0...,01060000206A7F000001000000010300000001000000DF...,drb,None,NaN,74297.0,74299.0,020401010101
2,2612920,797.8818,0.0373,8339.1431,0.4056,366215.4767,17.1041,3191.3462,4174.83,0.2170,...,99.5046,27.179,MULTILINESTRING Z ((531474.494 4696977.042 0.0...,01060000206A7F000001000000010300000001000000C1...,drb,None,NaN,74294.0,74299.0,020401010101


### Set Index to COMID
For easier search and error-free merging.  
Doc: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.set_index.html

In [57]:
# Set index to COMID
base_df.set_index('comid', inplace=True)
rest_df.set_index('comid', inplace=True)

### Sort Index
For performance.  
Docs: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sort_index.html

In [58]:
# Sort Index
base_df.sort_index(inplace=True)
rest_df.sort_index(inplace=True)

In [59]:
base_df.index

Int64Index([  1748535,   1748537,   1748539,   1748541,   1748543,   1748545,
              1748547,   1748549,   1748551,   1748553,
            ...
            932040361, 932040362, 932040363, 932040364, 932040365, 932040366,
            932040367, 932040368, 932040369, 932040370],
           dtype='int64', name='comid', length=19496)

In [60]:
rest_df.index

Int64Index([  1748535,   1748537,   1748539,   1748541,   1748543,   1748545,
              1748547,   1748549,   1748551,   1748553,
            ...
            932040361, 932040362, 932040363, 932040364, 932040365, 932040366,
            932040367, 932040368, 932040369, 932040370],
           dtype='int64', name='comid', length=19496)

### Confirm Differences between Base & Restoration

In [61]:
diff_df = base_df.tploadrate_total == rest_df.tploadrate_total
diff_df

comid
1748535      True
1748537      True
1748539      True
1748541      True
1748543      True
             ... 
932040366    True
932040367    True
932040368    True
932040369    True
932040370    True
Name: tploadrate_total, Length: 19496, dtype: bool

In [62]:
diff_df.value_counts()

True     19267
False      229
Name: tploadrate_total, dtype: int64

In [63]:
diff_df = base_df.tploadate_conc == rest_df.tploadate_conc
diff_df.value_counts()

True     18666
False      830
Name: tploadate_conc, dtype: int64

### Convert Nord, NordStop, Sub_FocusArea from Float to Nullable Integers
For improved performance, memory use, and compression.
Floats are generally problematic for any index or field used as an index.

Nord & NordStop contain null values, which by default Pandas interprets as a `numpy.nan` that is a `float64` dtype. This forces the entire series to `float64`.

Pandas can represent integer data with missing values (since v1.0) using `arrays.IntegerArray` and `dtype=pd.Int64Dtype()`. 

Docs:
- https://pandas.pydata.org/pandas-docs/stable/user_guide/integer_na.html
- https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.astype.html

In [64]:
base_df.sub_focusarea = base_df.sub_focusarea.astype(pd.Int64Dtype())
base_df.nord          = base_df.nord.astype(pd.Int64Dtype())
base_df.nordstop      = base_df.nordstop.astype(pd.Int64Dtype())

rest_df.sub_focusarea = rest_df.sub_focusarea.astype(pd.Int64Dtype())
rest_df.nord          = rest_df.nord.astype(pd.Int64Dtype())
rest_df.nordstop      = rest_df.nordstop.astype(pd.Int64Dtype()) 

In [65]:
base_df[base_df.sub_focusarea.notna()].head(3)

,tploadrate_total,tploadate_conc,tnloadrate_total,tnloadate_conc,tssloadrate_total,tssloadate_conc,catchment_hectares,watershed_hectares,tploadrate_total_ws,tnloadrate_total_ws,tssloadrate_total_ws,maflowv,geom,geom_catchment,cluster,fa_name,sub_focusarea,nord,nordstop,huc12
comid,,,,,,,,,,,,,,,,,,,,
2583353,177.7780,0.0281,2200.1500,0.3300,104612.1133,25.3955,486.6020,1723.41,0.1870,2.1958,168.9836,12.833,MULTILINESTRING Z ((510062.992 4547299.159 0.0...,01060000206A7F0000010000000103000000010000002D...,New Jersey Highlands,Paulins Kill,3,70054,70057,020401050102
2583369,3.8052,0.0802,38.6503,1.1833,10032.3886,82.1927,7.0144,1087.56,0.5157,7.6090,528.5280,7.826,MULTILINESTRING Z ((514598.774 4544239.887 0.0...,01060000206A7F00000100000001030000000100000018...,New Jersey Highlands,Paulins Kill,8,70152,70157,020401050102
2583379,47.4438,0.0609,1163.0701,1.0621,40602.0194,68.6833,114.2091,18845.46,0.3768,6.5716,424.9715,130.487,MULTILINESTRING Z ((514435.695 4544602.881 0.0...,01060000206A7F000001000000010300000001000000BA...,New Jersey Highlands,Paulins Kill,1,70059,70157,020401050102


### Convert String Objects to Categoricals
For improved performance, memory use, and compression.  
See https://anaconda.org/TomAugspurger/pandas-performance/notebook

Doc: https://pandas.pydata.org/pandas-docs/stable/user_guide/categorical.html

In [69]:
%%time
# Compare before and after
base_df.huc12.count()

CPU times: user 1.61 ms, sys: 98 µs, total: 1.71 ms
Wall time: 1.72 ms


19496

In [70]:
base_df.cluster = base_df.cluster.astype('category')
base_df.fa_name = base_df.fa_name.astype('category')
base_df.huc12   = base_df.huc12.astype('category')

rest_df.cluster = rest_df.cluster.astype('category')
rest_df.fa_name = rest_df.fa_name.astype('category')
rest_df.huc12   = rest_df.huc12.astype('category')

In [71]:
base_df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 19496 entries, 1748535 to 932040370
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   tploadrate_total      19496 non-null  float64 
 1   tploadate_conc        19496 non-null  float64 
 2   tnloadrate_total      19496 non-null  float64 
 3   tnloadate_conc        19496 non-null  float64 
 4   tssloadrate_total     19496 non-null  float64 
 5   tssloadate_conc       19496 non-null  float64 
 6   catchment_hectares    19496 non-null  float64 
 7   watershed_hectares    19496 non-null  float64 
 8   tploadrate_total_ws   19496 non-null  float64 
 9   tnloadrate_total_ws   19496 non-null  float64 
 10  tssloadrate_total_ws  19496 non-null  float64 
 11  maflowv               19496 non-null  float64 
 12  geom                  19266 non-null  geometry
 13  geom_catchment        19496 non-null  object  
 14  cluster               17358 non-null

In [72]:
%%time
# This is now an order of magnitude faster
base_df.huc12.count()

CPU times: user 323 µs, sys: 142 µs, total: 465 µs
Wall time: 343 µs


19496

In [73]:
%%time
base_df.huc12.unique()

CPU times: user 355 µs, sys: 35 µs, total: 390 µs
Wall time: 297 µs


['020401020302', '020401020305', '020401020402', '020401020303', '020401020306', ..., '020402060604', '020402060504', '020402060605', '020402060103', '020402050701']
Length: 484
Categories (484, object): ['020401010101', '020401010102', '020401010103', '020401010104', ..., '020403020406', '020403020407', '020403020408', '020403030101']

In [74]:
%%time
base_df.huc12.value_counts()

CPU times: user 1.37 ms, sys: 616 µs, total: 1.98 ms
Wall time: 1.44 ms


020403010704    310
020403010610    256
020403010406    253
020402060605    239
020402070203    231
               ... 
020402070605      2
020401060812      2
020403030101      2
020401020301      1
020401020401      1
Name: huc12, Length: 484, dtype: int64

### Correct Names

**Note on `ploadrate_total`, etc.**  
The microservice output says "tploadrate_total", however Mike suspected this is a mistake, and it actually is the local NHDplus **catchment annual load (kg/ha)**. We confirmed this by comparing to Model My Watershed subbasin modeling output for specific COMDIDs.
- The stream concentrations take into account the upstream watershed load, but this value is not returned. Mike calculated this when exporting the microservice results to PG for convenience, and saved as `tploadrate_total_ws` etc.

This code block assigns more accurate, shorter names to selected columns. All fields represent totals for the reach or catcment, so we are removing that from the name to shorten it.

Docs: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rename.html

In [75]:
base_df.rename(columns={'tploadrate_total':'tp_load',
                        'tploadate_conc':'tp_conc',
                        'tnloadrate_total':'tn_load',
                        'tnloadate_conc':'tn_conc',
                        'tssloadrate_total':'tss_load',
                        'tssloadate_conc':'tss_conc',
                        'tploadrate_total_ws':'tp_loadrate_ws',
                        'tnloadrate_total_ws':'tn_loadrate_ws',
                        'tssloadrate_total_ws':'tss_loadrate_ws',
                       },
               inplace=True,
              )
base_df.head(3)

,tp_load,tp_conc,tn_load,tn_conc,tss_load,tss_conc,catchment_hectares,watershed_hectares,tp_loadrate_ws,tn_loadrate_ws,tss_loadrate_ws,maflowv,geom,geom_catchment,cluster,fa_name,sub_focusarea,nord,nordstop,huc12
comid,,,,,,,,,,,,,,,,,,,,
1748535,881.4618,0.0226,10322.4299,0.2643,390433.1419,9.9983,6496.7052,6501.69,0.1357,1.5874,60.0509,43.699,MULTILINESTRING Z ((539681.467 4689416.166 0.0...,01060000206A7F00000100000001030000000100000021...,drb,NaN,<NA>,74914,74914,020401020302
1748537,296.6355,0.0297,3165.6081,0.3166,88090.7401,8.8103,1663.1712,1664.46,0.1784,1.9019,52.9245,11.189,MULTILINESTRING Z ((532825.185 4684387.302 0.0...,01060000206A7F000001000000010300000001000000FC...,drb,NaN,<NA>,74913,74913,020401020302
1748539,350.9217,0.0350,2816.4257,0.2808,117212.5160,11.6874,1639.4128,1640.70,0.2139,1.7164,71.4407,11.223,MULTILINESTRING Z ((524096.535 4677200.269 0.0...,01060000206A7F000001000000010300000001000000C2...,drb,NaN,<NA>,74921,74921,020401020305


In [76]:
rest_df.rename(columns={'tploadrate_total':'tp_load',
                        'tploadate_conc':'tp_conc',
                        'tnloadrate_total':'tn_load',
                        'tnloadate_conc':'tn_conc',
                        'tssloadrate_total':'tss_load',
                        'tssloadate_conc':'tss_conc',
                        'tploadrate_total_ws':'tp_loadrate_ws',
                        'tnloadrate_total_ws':'tn_loadrate_ws',
                        'tssloadrate_total_ws':'tss_loadrate_ws',
                       },
               inplace=True,
              )
rest_df.head(3)

,tp_load,tp_conc,tn_load,tn_conc,tss_load,tss_conc,catchment_hectares,watershed_hectares,tp_loadrate_ws,tn_loadrate_ws,tss_loadrate_ws,maflowv,geom,geom_catchment,cluster,fa_name,sub_focusarea,nord,nordstop,huc12
comid,,,,,,,,,,,,,,,,,,,,
1748535,881.4618,0.0226,10322.4299,0.2643,390433.1419,9.9983,6496.7052,6501.69,0.1357,1.5874,60.0509,43.699,MULTILINESTRING Z ((539681.467 4689416.166 0.0...,01060000206A7F00000100000001030000000100000021...,drb,NaN,<NA>,74914,74914,020401020302
1748537,296.6355,0.0297,3165.6081,0.3166,88090.7401,8.8103,1663.1712,1664.46,0.1784,1.9019,52.9245,11.189,MULTILINESTRING Z ((532825.185 4684387.302 0.0...,01060000206A7F000001000000010300000001000000FC...,drb,NaN,<NA>,74913,74913,020401020302
1748539,350.9217,0.0350,2816.4257,0.2808,117212.5160,11.6874,1639.4128,1640.70,0.2139,1.7164,71.4407,11.223,MULTILINESTRING Z ((524096.535 4677200.269 0.0...,01060000206A7F000001000000010300000001000000C2...,drb,NaN,<NA>,74921,74921,020401020305


## Convert to Reach-Specific GeoDataFrame

A GeoPandas DataFrame can only have one geometry column, and the auto-import defaulted to the reach polyline. So create a new DataFrame and drop catchment-specific columns.

In [106]:
base_df_reach = base_df.drop(['tp_load',
                              'tn_load',
                              'tss_load',
                              'tp_loadrate_ws',
                              'tn_loadrate_ws',
                              'tss_loadrate_ws',
                              'geom_catchment',
                             ], axis=1)
rest_df_reach = rest_df.drop(['tp_load',
                              'tn_load',
                              'tss_load',
                              'tp_loadrate_ws',
                              'tn_loadrate_ws',
                              'tss_loadrate_ws',
                              'geom_catchment',
                             ], axis=1)

In [107]:
base_df_reach.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 19496 entries, 1748535 to 932040370
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype   
---  ------              --------------  -----   
 0   tp_conc             19496 non-null  float64 
 1   tn_conc             19496 non-null  float64 
 2   tss_conc            19496 non-null  float64 
 3   catchment_hectares  19496 non-null  float64 
 4   watershed_hectares  19496 non-null  float64 
 5   maflowv             19496 non-null  float64 
 6   geom                19266 non-null  geometry
 7   cluster             17358 non-null  category
 8   fa_name             186 non-null    category
 9   sub_focusarea       186 non-null    Int64   
 10  nord                18870 non-null  Int64   
 11  nordstop            18844 non-null  Int64   
 12  huc12               19496 non-null  category
dtypes: Int64(3), category(3), float64(6), geometry(1)
memory usage: 1.8 MB


In [108]:
rest_df_reach.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 19496 entries, 1748535 to 932040370
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype   
---  ------              --------------  -----   
 0   tp_conc             19496 non-null  float64 
 1   tn_conc             19496 non-null  float64 
 2   tss_conc            19496 non-null  float64 
 3   catchment_hectares  19496 non-null  float64 
 4   watershed_hectares  19496 non-null  float64 
 5   maflowv             19496 non-null  float64 
 6   geom                19266 non-null  geometry
 7   cluster             17358 non-null  category
 8   fa_name             186 non-null    category
 9   sub_focusarea       186 non-null    Int64   
 10  nord                18870 non-null  Int64   
 11  nordstop            18844 non-null  Int64   
 12  huc12               19496 non-null  category
dtypes: Int64(3), category(3), float64(6), geometry(1)
memory usage: 1.8 MB


In [109]:
base_df_reach

,tp_conc,tn_conc,tss_conc,catchment_hectares,watershed_hectares,maflowv,geom,cluster,fa_name,sub_focusarea,nord,nordstop,huc12
comid,,,,,,,,,,,,,
1748535,0.0226,0.2643,9.9983,6496.7052,6501.69,43.699,MULTILINESTRING Z ((539681.467 4689416.166 0.0...,drb,NaN,<NA>,74914,74914,020401020302
1748537,0.0297,0.3166,8.8103,1663.1712,1664.46,11.189,MULTILINESTRING Z ((532825.185 4684387.302 0.0...,drb,NaN,<NA>,74913,74913,020401020302
1748539,0.0350,0.2808,11.6874,1639.4128,1640.70,11.223,MULTILINESTRING Z ((524096.535 4677200.269 0.0...,drb,NaN,<NA>,74921,74921,020401020305
1748541,0.0245,0.2703,10.2217,3013.8348,12912.30,86.528,MULTILINESTRING Z ((533110.692 4677277.923 0.0...,drb,NaN,<NA>,74911,74915,020401020302
1748543,0.0307,0.2661,10.8301,1151.0990,5232.87,35.389,MULTILINESTRING Z ((526672.244 4673109.451 0.0...,drb,NaN,<NA>,74920,74922,020401020305
...,...,...,...,...,...,...,...,...,...,...,...,...,...
932040366,-9999.0000,-9999.0000,-9999.0000,2124.7248,2720941.47,17802.923,None,drb,NaN,<NA>,65070,76964,020402060103
932040367,-9999.0000,-9999.0000,-9999.0000,788.7859,2717821.26,17788.281,None,drb,NaN,<NA>,65079,76964,020402060103
932040368,-9999.0000,-9999.0000,-9999.0000,265.0275,2716120.08,17780.448,None,drb,NaN,<NA>,65080,76960,020402060103


### Confirm Differences between Base & Restoration

In [110]:
diff_df = base_df_reach.tp_conc == rest_df_reach.tp_conc
diff_df.value_counts()

True     18666
False      830
Name: tp_conc, dtype: int64

### Replace -9999.0 Concentrations with NaN

**Question:** should we set these to zero rather than NaN?

In [111]:
base_df_reach.tp_conc[base_df_reach.tp_conc==-9999.0].count()

2673

In [112]:
rest_df_reach.tp_conc[rest_df_reach.tp_conc==-9999.0].count()

2673

In [113]:
import numpy as np
base_df_reach.replace(to_replace=-9999.0, value=np.NaN, inplace=True)
rest_df_reach.replace(to_replace=-9999.0, value=np.NaN, inplace=True)

In [114]:
base_df_reach.tp_conc[base_df_reach.tp_conc==-9999.0].count()

0

In [115]:
rest_df_reach.tp_conc[rest_df_reach.tp_conc==-9999.0].count()

0

In [116]:
rest_df_reach

,tp_conc,tn_conc,tss_conc,catchment_hectares,watershed_hectares,maflowv,geom,cluster,fa_name,sub_focusarea,nord,nordstop,huc12
comid,,,,,,,,,,,,,
1748535,0.0226,0.2643,9.9983,6496.7052,6501.69,43.699,MULTILINESTRING Z ((539681.467 4689416.166 0.0...,drb,NaN,<NA>,74914,74914,020401020302
1748537,0.0297,0.3166,8.8103,1663.1712,1664.46,11.189,MULTILINESTRING Z ((532825.185 4684387.302 0.0...,drb,NaN,<NA>,74913,74913,020401020302
1748539,0.0350,0.2808,11.6874,1639.4128,1640.70,11.223,MULTILINESTRING Z ((524096.535 4677200.269 0.0...,drb,NaN,<NA>,74921,74921,020401020305
1748541,0.0245,0.2703,10.2217,3013.8348,12912.30,86.528,MULTILINESTRING Z ((533110.692 4677277.923 0.0...,drb,NaN,<NA>,74911,74915,020401020302
1748543,0.0307,0.2661,10.8301,1151.0990,5232.87,35.389,MULTILINESTRING Z ((526672.244 4673109.451 0.0...,drb,NaN,<NA>,74920,74922,020401020305
...,...,...,...,...,...,...,...,...,...,...,...,...,...
932040366,NaN,NaN,NaN,2124.7248,2720941.47,17802.923,None,drb,NaN,<NA>,65070,76964,020402060103
932040367,NaN,NaN,NaN,788.7859,2717821.26,17788.281,None,drb,NaN,<NA>,65079,76964,020402060103
932040368,NaN,NaN,NaN,265.0275,2716120.08,17780.448,None,drb,NaN,<NA>,65080,76960,020402060103


In [117]:
diff_df = base_df_reach.tp_conc == rest_df_reach.tp_conc
diff_df.value_counts()

True     15993
False     3503
Name: tp_conc, dtype: int64

**NOTE: NaN comparisons produce FALSE**

## Convert to Catchment-Specific GeoDataFrame

A GeoPandas DataFrame can only have one geometry column, and the auto-import defaulted to the reach polyline. So create a new DataFrame and drop catchment-specific columns.

In [118]:
base_df_catch = base_df.drop(['tp_conc',
                              'tn_conc',
                              'tss_conc',
                              'geom',
                             ], axis=1)
rest_df_catch = rest_df.drop(['tp_conc',
                              'tn_conc',
                              'tss_conc',
                              'geom',
                             ], axis=1)

In [119]:
base_df_catch.head()

,tp_load,tn_load,tss_load,catchment_hectares,watershed_hectares,tp_loadrate_ws,tn_loadrate_ws,tss_loadrate_ws,maflowv,geom_catchment,cluster,fa_name,sub_focusarea,nord,nordstop,huc12
comid,,,,,,,,,,,,,,,,
1748535,881.4618,10322.4299,390433.1419,6496.7052,6501.69,0.1357,1.5874,60.0509,43.699,01060000206A7F00000100000001030000000100000021...,drb,NaN,<NA>,74914,74914,020401020302
1748537,296.6355,3165.6081,88090.7401,1663.1712,1664.46,0.1784,1.9019,52.9245,11.189,01060000206A7F000001000000010300000001000000FC...,drb,NaN,<NA>,74913,74913,020401020302
1748539,350.9217,2816.4257,117212.5160,1639.4128,1640.70,0.2139,1.7164,71.4407,11.223,01060000206A7F000001000000010300000001000000C2...,drb,NaN,<NA>,74921,74921,020401020305
1748541,517.2223,5245.9416,201648.7618,3013.8348,12912.30,0.1467,1.6186,61.2103,86.528,01060000206A7F00000100000001030000000100000051...,drb,NaN,<NA>,74911,74915,020401020302
1748543,289.1315,2478.5764,108737.6061,1151.0990,5232.87,0.1855,1.6081,65.4499,35.389,01060000206A7F000001000000010300000001000000A1...,drb,NaN,<NA>,74920,74922,020401020305


In [120]:
rest_df_catch.head()

,tp_load,tn_load,tss_load,catchment_hectares,watershed_hectares,tp_loadrate_ws,tn_loadrate_ws,tss_loadrate_ws,maflowv,geom_catchment,cluster,fa_name,sub_focusarea,nord,nordstop,huc12
comid,,,,,,,,,,,,,,,,
1748535,881.4618,10322.4299,390433.1419,6496.7052,6501.69,0.1357,1.5874,60.0509,43.699,01060000206A7F00000100000001030000000100000021...,drb,NaN,<NA>,74914,74914,020401020302
1748537,296.6355,3165.6081,88090.7401,1663.1712,1664.46,0.1784,1.9019,52.9245,11.189,01060000206A7F000001000000010300000001000000FC...,drb,NaN,<NA>,74913,74913,020401020302
1748539,350.9217,2816.4257,117212.5160,1639.4128,1640.70,0.2139,1.7164,71.4407,11.223,01060000206A7F000001000000010300000001000000C2...,drb,NaN,<NA>,74921,74921,020401020305
1748541,517.2223,5245.9416,201648.7618,3013.8348,12912.30,0.1467,1.6186,61.2103,86.528,01060000206A7F00000100000001030000000100000051...,drb,NaN,<NA>,74911,74915,020401020302
1748543,289.1315,2478.5764,108737.6061,1151.0990,5232.87,0.1855,1.6081,65.4499,35.389,01060000206A7F000001000000010300000001000000A1...,drb,NaN,<NA>,74920,74922,020401020305


### Confirm Differences between Base & Restoration

In [123]:
diff_df = base_df_catch.tp_load == rest_df_catch.tp_load
diff_df.value_counts()

True     19267
False      229
Name: tp_load, dtype: int64

### Read Database Again To Set Catchment Geometry
This reruns the `gpd.read_postgis()` function with the optional argument `geom_col="geom_catchment"`, as a workaround for converting the catchment boundaries to geometry datatypes.
Also only selects the two required fields to speed up the response.

In [124]:
%%time
# get catchment geometry
base_catch = 'SELECT comid, geom_catchment FROM wikiwtershedoutputs.{}'.format(tablename_base)
rest_catch = 'SELECT comid, geom_catchment FROM wikiwtershedoutputs.{}'.format(tablename_rest)

base_df_catch_geom = gpd.read_postgis(base_catch, con, geom_col="geom_catchment")
rest_df_catch_geom = gpd.read_postgis(rest_catch, con, geom_col="geom_catchment")

CPU times: user 3.99 s, sys: 725 ms, total: 4.72 s
Wall time: 30.8 s


In [125]:
base_df_catch_geom.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 19496 entries, 0 to 19495
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   comid           19496 non-null  int64   
 1   geom_catchment  19496 non-null  geometry
dtypes: geometry(1), int64(1)
memory usage: 304.8 KB


In [126]:
base_df_catch_geom.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 19496 entries, 0 to 19495
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   comid           19496 non-null  int64   
 1   geom_catchment  19496 non-null  geometry
dtypes: geometry(1), int64(1)
memory usage: 304.8 KB


In [127]:
# Set index to comid
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.set_index.html
base_df_catch_geom.set_index('comid', inplace=True)
rest_df_catch_geom.set_index('comid', inplace=True)

In [128]:
base_df_catch_geom.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 19496 entries, 2612952 to 8409235
Data columns (total 1 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   geom_catchment  19496 non-null  geometry
dtypes: geometry(1)
memory usage: 304.6 KB


### Replace `geom_catchment` Object with Geometry DType

In [129]:
# merge
base_df_catch['geom_catchment'] = base_df_catch_geom['geom_catchment']
rest_df_catch['geom_catchment'] = rest_df_catch_geom['geom_catchment']

In [130]:
base_df_catch.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 19496 entries, 1748535 to 932040370
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype   
---  ------              --------------  -----   
 0   tp_load             19496 non-null  float64 
 1   tn_load             19496 non-null  float64 
 2   tss_load            19496 non-null  float64 
 3   catchment_hectares  19496 non-null  float64 
 4   watershed_hectares  19496 non-null  float64 
 5   tp_loadrate_ws      19496 non-null  float64 
 6   tn_loadrate_ws      19496 non-null  float64 
 7   tss_loadrate_ws     19496 non-null  float64 
 8   maflowv             19496 non-null  float64 
 9   geom_catchment      19496 non-null  geometry
 10  cluster             17358 non-null  category
 11  fa_name             186 non-null    category
 12  sub_focusarea       186 non-null    Int64   
 13  nord                18870 non-null  Int64   
 14  nordstop            18844 non-null  Int64   
 15  huc12             

In [131]:
base_df_catch.head(3)

,tp_load,tn_load,tss_load,catchment_hectares,watershed_hectares,tp_loadrate_ws,tn_loadrate_ws,tss_loadrate_ws,maflowv,geom_catchment,cluster,fa_name,sub_focusarea,nord,nordstop,huc12
comid,,,,,,,,,,,,,,,,
1748535,881.4618,10322.4299,390433.1419,6496.7052,6501.69,0.1357,1.5874,60.0509,43.699,"MULTIPOLYGON (((535287.346 4678015.900, 535140...",drb,NaN,<NA>,74914,74914,020401020302
1748537,296.6355,3165.6081,88090.7401,1663.1712,1664.46,0.1784,1.9019,52.9245,11.189,"MULTIPOLYGON (((532639.560 4678753.867, 532610...",drb,NaN,<NA>,74913,74913,020401020302
1748539,350.9217,2816.4257,117212.5160,1639.4128,1640.70,0.2139,1.7164,71.4407,11.223,"MULTIPOLYGON (((525043.074 4672557.985, 525013...",drb,NaN,<NA>,74921,74921,020401020305


In [132]:
rest_df_catch.head(3)

,tp_load,tn_load,tss_load,catchment_hectares,watershed_hectares,tp_loadrate_ws,tn_loadrate_ws,tss_loadrate_ws,maflowv,geom_catchment,cluster,fa_name,sub_focusarea,nord,nordstop,huc12
comid,,,,,,,,,,,,,,,,
1748535,881.4618,10322.4299,390433.1419,6496.7052,6501.69,0.1357,1.5874,60.0509,43.699,"MULTIPOLYGON (((535287.346 4678015.900, 535140...",drb,NaN,<NA>,74914,74914,020401020302
1748537,296.6355,3165.6081,88090.7401,1663.1712,1664.46,0.1784,1.9019,52.9245,11.189,"MULTIPOLYGON (((532639.560 4678753.867, 532610...",drb,NaN,<NA>,74913,74913,020401020302
1748539,350.9217,2816.4257,117212.5160,1639.4128,1640.70,0.2139,1.7164,71.4407,11.223,"MULTIPOLYGON (((525043.074 4672557.985, 525013...",drb,NaN,<NA>,74921,74921,020401020305


## Add Units

Loads are in kg/y.  
Concentrations are in mg/L.

**INCOMPLETE**

Options for the future:
- https://stackoverflow.com/questions/14688306/adding-meta-information-metadata-to-pandas-dataframe
- https://pandas.pydata.org/docs/reference/api/pandas.Series.attrs.html
- https://stackoverflow.com/questions/39419178/how-can-i-manage-units-in-pandas-data
- https://github.com/hgrecco/pint
- https://towardsdatascience.com/add-metadata-to-your-dataset-using-apache-parquet-75360d2073bd

In [ ]:
#base_df.attrs

In [ ]:
#base_df.tp_load.attrs

## Get the Protection Projects "Avoided Loads"

Returns the results for the protection projects uploaded into FieldDoc. Protection projects were exploaded from multi-polygon to polygon then intersected with NHDplus COMID.

The avoided load is taken as the Developed load - the Forested load. Assumes 100% developement to Medium Intensity (NLCD class 23). Only the average forested loading rate is provided in this table (averagess NLCD codes 41,42,43) in order to make the table simpler, however each land cover class' loading rate is explicitly used in the "avoided" load calculation.

Primary Key
- practice_i = FieldDoc (FD) primary key ID
- rn = Row number for number of polygons in the multipolygon submitted to FD as a project
- comid = NHDplus COMID intesecting with the protection polygon

SQL:
```sql
CONSTRAINT pk_protection_lbsavoided_fd PRIMARY KEY (practice_i, rn, comid)
```

**TO DO** in the future:
- Implement a Pandas/GeoPandas multi-index:  
https://pandas.pydata.org/pandas-docs/stable/user_guide/advanced.html

In [133]:
%%time

db_connection_url = "postgresql://{}:{}@{}:{}/{}".format(_user[0], _password[0], _host[0], _port, _database[0])
con = create_engine(db_connection_url)  

tablename_prot = 'protection_lbsavoided_fd'

prot_select = 'SELECT * FROM datapolassess.{}'.format(tablename_prot)

prot_proj_df = gpd.read_postgis(prot_select, con)

CPU times: user 52.3 ms, sys: 14.8 ms, total: 67 ms
Wall time: 1.69 s


In [134]:
prot_proj_df.head()

,practice_i,practice_n,rn,comid,practice_t,practice_d,project_na,project_st,creator_na,program_id,...,parcel_devtssload_lbyr,parcel_foretssload_lbyr,parcel_tssload_lbyr_avoided,parcel_devtnload_lbyr,parcel_foretnload_lbyr,parcel_tnload_lbyr_avoided,parcel_devtpload_lbyr,parcel_foretpload_lbyr,parcel_tpload_lbyr_avoided,geom
0,5289.0,Bear Creek,1,4185445,Fee acquisition,Acquisition,Bear Creek - Crystal Lake,complete,Dawn Gorham,5.0,...,175779.645,9187.118,166592.527,205.600,17.768,187.832,76.994,4.230,72.764,"MULTIPOLYGON (((-75.79484 41.17775, -75.79468 ..."
1,5289.0,Bear Creek,1,4185461,Fee acquisition,Acquisition,Bear Creek - Crystal Lake,complete,Dawn Gorham,5.0,...,159240.637,8322.707,150917.930,186.255,16.096,170.159,69.750,3.832,65.918,"MULTIPOLYGON (((-75.78786 41.18276, -75.78756 ..."
2,5289.0,Bear Creek,1,4185483,Fee acquisition,Acquisition,Bear Creek - Crystal Lake,complete,Dawn Gorham,5.0,...,235688.406,12318.247,223370.159,275.672,23.823,251.849,103.235,5.672,97.563,"MULTIPOLYGON (((-75.81161 41.17841, -75.81176 ..."
3,5289.0,Bear Creek,1,4185485,Fee acquisition,Acquisition,Bear Creek - Crystal Lake,complete,Dawn Gorham,5.0,...,48109.916,2514.463,45595.453,56.272,4.863,51.409,21.073,1.158,19.915,"MULTIPOLYGON (((-75.80381 41.17354, -75.80405 ..."
4,5289.0,Bear Creek,1,4185505,Fee acquisition,Acquisition,Bear Creek - Crystal Lake,complete,Dawn Gorham,5.0,...,40026.507,2083.105,37943.402,34.930,3.776,31.154,16.804,0.944,15.860,"MULTIPOLYGON (((-75.79805 41.17363, -75.80381 ..."


In [135]:
prot_proj_df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 269 entries, 0 to 268
Data columns (total 33 columns):
 #   Column                        Non-Null Count  Dtype   
---  ------                        --------------  -----   
 0   practice_i                    269 non-null    float64 
 1   practice_n                    269 non-null    object  
 2   rn                            269 non-null    int64   
 3   comid                         269 non-null    int64   
 4   practice_t                    269 non-null    object  
 5   practice_d                    44 non-null     object  
 6   project_na                    269 non-null    object  
 7   project_st                    269 non-null    object  
 8   creator_na                    269 non-null    object  
 9   program_id                    269 non-null    float64 
 10  program_na                    269 non-null    object  
 11  created                       269 non-null    object  
 12  modified                      269 non-null

### Convert DataTypes

- Some `float64` columns should be converted to the nullable integers (`pd.Int64Dtype()`).
- Many string `object` columns can be converted to categoricals.
- Date and time string `object` columns should be parsed to DateTimes (`datetime64`).

Both of these improve performance, memory use, and compression. Floats are generally problematic for any index or field used as an index.

Docs:
- https://anaconda.org/TomAugspurger/pandas-performance/notebook
- https://pandas.pydata.org/pandas-docs/stable/user_guide/integer_na.html
- https://pandas.pydata.org/pandas-docs/stable/user_guide/categorical.html
- https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html

In [136]:
# Convert IDs to nullable integers
prot_proj_df.practice_i = prot_proj_df.practice_i.astype(pd.Int64Dtype())
prot_proj_df.program_id = prot_proj_df.program_id.astype(pd.Int64Dtype())

In [137]:
# Convert string Objects to Categoricals
prot_proj_df.practice_n = prot_proj_df.practice_n.astype('category')
prot_proj_df.practice_t = prot_proj_df.practice_t.astype('category')
prot_proj_df.practice_d = prot_proj_df.practice_d.astype('category')
prot_proj_df.project_na = prot_proj_df.project_na.astype('category')
prot_proj_df.project_st = prot_proj_df.project_st.astype('category')
prot_proj_df.creator_na = prot_proj_df.creator_na.astype('category')
prot_proj_df.program_na = prot_proj_df.program_na.astype('category')
prot_proj_df.huc12      = prot_proj_df.huc12.astype('category')

In [138]:
# Convert date string Objects to DateTime
prot_proj_df.created  = pd.to_datetime(prot_proj_df.created, infer_datetime_format=True)
prot_proj_df.modified = pd.to_datetime(prot_proj_df.modified, infer_datetime_format=True)

In [139]:
prot_proj_df.iloc[:, 8:17].head()

,creator_na,program_id,program_na,created,modified,practice_url,project_url,huc12,area_acres
0,Dawn Gorham,5,Delaware River Watershed Protection Fund - For...,2019-01-04 19:03:54.420311,2019-11-15 20:56:34.050925,https://www.fielddoc.org/practices/5289,https://www.fielddoc.org/projects/2665,020401060203,84.609
1,Dawn Gorham,5,Delaware River Watershed Protection Fund - For...,2019-01-04 19:03:54.420311,2019-11-15 20:56:34.050925,https://www.fielddoc.org/practices/5289,https://www.fielddoc.org/projects/2665,020401060203,76.648
2,Dawn Gorham,5,Delaware River Watershed Protection Fund - For...,2019-01-04 19:03:54.420311,2019-11-15 20:56:34.050925,https://www.fielddoc.org/practices/5289,https://www.fielddoc.org/projects/2665,020401060203,113.445
3,Dawn Gorham,5,Delaware River Watershed Protection Fund - For...,2019-01-04 19:03:54.420311,2019-11-15 20:56:34.050925,https://www.fielddoc.org/practices/5289,https://www.fielddoc.org/projects/2665,020401060203,23.157
4,Dawn Gorham,5,Delaware River Watershed Protection Fund - For...,2019-01-04 19:03:54.420311,2019-11-15 20:56:34.050925,https://www.fielddoc.org/practices/5289,https://www.fielddoc.org/projects/2665,020401060204,18.881


In [140]:
prot_proj_df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 269 entries, 0 to 268
Data columns (total 33 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   practice_i                    269 non-null    Int64         
 1   practice_n                    269 non-null    category      
 2   rn                            269 non-null    int64         
 3   comid                         269 non-null    int64         
 4   practice_t                    269 non-null    category      
 5   practice_d                    44 non-null     category      
 6   project_na                    269 non-null    category      
 7   project_st                    269 non-null    category      
 8   creator_na                    269 non-null    category      
 9   program_id                    269 non-null    Int64         
 10  program_na                    269 non-null    category      
 11  created                 

## Get the Restoration Projects intersected by COMID

The FieldDoc primary key is `practice_i`, but we subdivide by `comid`, creating a multi-index.

SQL for the Primary Key:
```sql
CONSTRAINT pk_cache_restoration_lbsreduced PRIMARY KEY (comid, practice_i)
```

**TO DO** in the future:
- Implement a Pandas/GeoPandas multi-index:  
https://pandas.pydata.org/pandas-docs/stable/user_guide/advanced.html

In [141]:
%%time
tablename_rest_projects = 'restoration_lbsreduced'
rest_proj_select = 'SELECT * FROM datapolassess.{}'.format(tablename_rest_projects)
rest_proj_df = gpd.read_postgis(rest_proj_select, con)

ProgrammingError: (psycopg2.errors.UndefinedTable) relation "datapolassess.restoration_lbsreduced" does not exist
LINE 1: SELECT * FROM datapolassess.restoration_lbsreduced
                      ^

[SQL: SELECT * FROM datapolassess.restoration_lbsreduced]
(Background on this error at: https://sqlalche.me/e/14/f405)

In [8]:
rest_proj_df.head(3)

,comid,practice_id,site_id,practice_type,tn_reduced_lbs,tp_reduced_lbs,tss_reduced_lbs,practice_name,practice_description,project_name,project_status,creator_name,program_id,program_name,created_on,modified_on,practice_url,project_url,geom
0,2583213,5396,2864,Forest Buffer,2.8553,0.6454,331.0068,Forest buffer,None,EZG #53363 Restoring Paulins Kill Floodplain F...,active,Kristine Rogers,1,Delaware River Restoration Fund,2019-01-24,2021-07-23,https://www.fielddoc.org/practices/5396,https://www.fielddoc.org/projects/2737,"MULTIPOLYGON (((522215.487 4552852.203, 522202..."
1,2583213,5399,2865,Forest Buffer,14.1300,3.4400,1438.7700,Forest buffer,None,EZG #53363 Restoring Paulins Kill Floodplain F...,active,Kristine Rogers,1,Delaware River Restoration Fund,2019-01-24,2021-08-10,https://www.fielddoc.org/practices/5399,https://www.fielddoc.org/projects/2737,"MULTIPOLYGON (((522234.173 4553442.738, 522208..."
2,2583231,5396,2864,Forest Buffer,33.6847,7.6146,3904.9832,Forest buffer,None,EZG #53363 Restoring Paulins Kill Floodplain F...,active,Kristine Rogers,1,Delaware River Restoration Fund,2019-01-24,2021-07-23,https://www.fielddoc.org/practices/5396,https://www.fielddoc.org/projects/2737,"MULTIPOLYGON (((522215.487 4552852.203, 522202..."


In [9]:
rest_proj_df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 755 entries, 0 to 754
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   comid                 755 non-null    int64   
 1   practice_id           755 non-null    int64   
 2   site_id               755 non-null    int64   
 3   practice_type         755 non-null    object  
 4   tn_reduced_lbs        755 non-null    float64 
 5   tp_reduced_lbs        755 non-null    float64 
 6   tss_reduced_lbs       755 non-null    float64 
 7   practice_name         755 non-null    object  
 8   practice_description  485 non-null    object  
 9   project_name          755 non-null    object  
 10  project_status        755 non-null    object  
 11  creator_name          755 non-null    object  
 12  program_id            755 non-null    int64   
 13  program_name          755 non-null    object  
 14  created_on            755 non-null    object  
 15

### Convert DataTypes

As for Project Projects, with rational and methods described above.

In [225]:
# Convert IDs to nullable integers
rest_proj_df.practice_id = rest_proj_df.practice_id.astype(pd.Int64Dtype())
rest_proj_df.program_id = rest_proj_df.program_id.astype(pd.Int64Dtype())
rest_proj_df.site_id    = rest_proj_df.site_id.astype(pd.Int64Dtype())

In [226]:
# Convert string Objects to Categoricals
rest_proj_df.practice_name = rest_proj_df.practice_name.astype('category')
rest_proj_df.practice_type = rest_proj_df.practice_type.astype('category')
rest_proj_df.practice_description = rest_proj_df.practice_description.astype('category')
rest_proj_df.project_namae = rest_proj_df.project_name.astype('category')
rest_proj_df.project_status = rest_proj_df.project_status.astype('category')
rest_proj_df.creator_name = rest_proj_df.creator_name.astype('category')
rest_proj_df.program_name = rest_proj_df.program_name.astype('category')
# rest_proj_df.huc12      = rest_proj_df.huc12.astype('category')

C:\Users\mcamp\Anaconda3\envs\WikiSRAT\lib\site-packages\geopandas\geodataframe.py:182: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  super(GeoDataFrame, self).__setattr__(attr, val)


In [227]:
# Convert date string Objects to DateTime
rest_proj_df.created_on  = pd.to_datetime(rest_proj_df.created_on, infer_datetime_format=True)
rest_proj_df.modified_on = pd.to_datetime(rest_proj_df.modified_on, infer_datetime_format=True)

In [228]:
rest_proj_df.head(3)

,comid,practice_id,site_id,practice_type,tn_reduced_lbs,tp_reduced_lbs,tss_reduced_lbs,practice_name,practice_description,project_name,project_status,creator_name,program_id,program_name,created_on,modified_on,practice_url,project_url,geom
0,2583213,5396,2864,Forest Buffer,2.8553,0.6454,331.0068,Forest buffer,NaN,EZG #53363 Restoring Paulins Kill Floodplain F...,active,Kristine Rogers,1,Delaware River Restoration Fund,2019-01-24,2021-07-23,https://www.fielddoc.org/practices/5396,https://www.fielddoc.org/projects/2737,"MULTIPOLYGON (((522215.487 4552852.203, 522202..."
1,2583213,5399,2865,Forest Buffer,14.1300,3.4400,1438.7700,Forest buffer,NaN,EZG #53363 Restoring Paulins Kill Floodplain F...,active,Kristine Rogers,1,Delaware River Restoration Fund,2019-01-24,2021-08-10,https://www.fielddoc.org/practices/5399,https://www.fielddoc.org/projects/2737,"MULTIPOLYGON (((522234.173 4553442.738, 522208..."
2,2583231,5396,2864,Forest Buffer,33.6847,7.6146,3904.9832,Forest buffer,NaN,EZG #53363 Restoring Paulins Kill Floodplain F...,active,Kristine Rogers,1,Delaware River Restoration Fund,2019-01-24,2021-07-23,https://www.fielddoc.org/practices/5396,https://www.fielddoc.org/projects/2737,"MULTIPOLYGON (((522215.487 4552852.203, 522202..."


In [229]:
rest_proj_df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 755 entries, 0 to 754
Data columns (total 19 columns):
comid                   755 non-null int64
practice_id             755 non-null Int64
site_id                 755 non-null Int64
practice_type           755 non-null category
tn_reduced_lbs          755 non-null float64
tp_reduced_lbs          755 non-null float64
tss_reduced_lbs         755 non-null float64
practice_name           755 non-null category
practice_description    485 non-null category
project_name            755 non-null object
project_status          755 non-null category
creator_name            755 non-null category
program_id              755 non-null Int64
program_name            755 non-null category
created_on              755 non-null datetime64[ns]
modified_on             755 non-null datetime64[ns]
practice_url            755 non-null object
project_url             755 non-null object
geom                    755 non-null geometry
dtypes: Int64(3), cate

## Get the Point Sources by NPDES ID and COMID

In [142]:
%%time
tablename_pointsource = 'ms_pointsource_drb_12_13_18'
pointsource_select = 'SELECT * FROM wikiwtershed.{}'.format(tablename_pointsource)
point_source_df = gpd.read_postgis(pointsource_select, con)

CPU times: user 51.3 ms, sys: 4.79 ms, total: 56.1 ms
Wall time: 790 ms


In [143]:
point_source_df.head()

,ogc_fid,geom,npdes_id,city,state,latitude,longitude,huc12,avg_n_conc,lbsn_yr,mgd,avgpconc,lbsp_yr,kgn_yr,kgp_yr,facilityname,comid
0,1,POINT (414070.808 4469871.626),PA0033995,BERN TWP,PA,40.375000,-76.012222,020402030409,0.191,84.591269,0.163050,0.191,1325.042759,38.369923,601.028795,COUNTY OF BERKS WWTP,4783187
1,1,POINT (450085.743 4495200.414),PA0051811,SOUTH WHITEHALL1TWP,PA,40.606111,-75.590000,020401060703,35.000,32.011082,0.000300,35.000,2.733729,14.519971,1.239998,LEHIGH COUNTY AUTH,4187751
2,1,POINT (443086.168 4439655.703),1594403,WEST VINCENT TWP,PA,40.105277,-75.667777,020402031003,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,MATTHEWS MEADOWS STP,4781791
3,1,POINT (444600.842 4439582.877),1592417,WEST VINCENT TWP,PA,40.104722,-75.650000,020402031003,0.000,0.000000,0.000000,0.000,NaN,0.000000,NaN,SAINT STEPHEN'S GREENE STP,4782621
4,1,POINT (499574.949 4511288.150),NJ0065196,Township of Washington,NJ,40.752548,-75.005035,020401050401,7.175,155.557987,0.000677,7.175,5.599735,70.559859,2.539995,390 RT 57,2588253


In [144]:
point_source_df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 812 entries, 0 to 811
Data columns (total 17 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   ogc_fid       812 non-null    int64   
 1   geom          812 non-null    geometry
 2   npdes_id      812 non-null    object  
 3   city          789 non-null    object  
 4   state         812 non-null    object  
 5   latitude      812 non-null    float64 
 6   longitude     812 non-null    float64 
 7   huc12         812 non-null    object  
 8   avg_n_conc    811 non-null    float64 
 9   lbsn_yr       811 non-null    float64 
 10  mgd           812 non-null    float64 
 11  avgpconc      811 non-null    float64 
 12  lbsp_yr       809 non-null    float64 
 13  kgn_yr        811 non-null    float64 
 14  kgp_yr        809 non-null    float64 
 15  facilityname  812 non-null    object  
 16  comid         812 non-null    int64   
dtypes: float64(9), geometry(1), int64(2), object(5

### Convert DataTypes

As for Project Projects, with rational and methods described above.

In [145]:
# Convert string Objects to Categoricals
point_source_df.npdes_id = point_source_df.npdes_id.astype('category')
point_source_df.city     = point_source_df.city.astype('category')
point_source_df.state    = point_source_df.state.astype('category')
point_source_df.huc12 = point_source_df.huc12.astype('category')
point_source_df.facilityname = point_source_df.facilityname.astype('category')

In [146]:
point_source_df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 812 entries, 0 to 811
Data columns (total 17 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   ogc_fid       812 non-null    int64   
 1   geom          812 non-null    geometry
 2   npdes_id      812 non-null    category
 3   city          789 non-null    category
 4   state         812 non-null    category
 5   latitude      812 non-null    float64 
 6   longitude     812 non-null    float64 
 7   huc12         812 non-null    category
 8   avg_n_conc    811 non-null    float64 
 9   lbsn_yr       811 non-null    float64 
 10  mgd           812 non-null    float64 
 11  avgpconc      811 non-null    float64 
 12  lbsp_yr       809 non-null    float64 
 13  kgn_yr        811 non-null    float64 
 14  kgp_yr        809 non-null    float64 
 15  facilityname  812 non-null    category
 16  comid         812 non-null    int64   
dtypes: category(5), float64(9), geometry(1), int64

### Set Index to `npdes_id`

`npdes_id` appears to be the Primary Key

In [147]:
# Confirm that the `npdes_id` hase one unique value per data row.
point_source_df.npdes_id.unique()

['PA0033995', 'PA0051811', '1594403', '1592417', 'NJ0065196', ..., 'NJ0022985', 'PA0056081', 'PA0036081', 'PA0026638', 'NJ0104388']
Length: 812
Categories (812, object): ['1503410', '1505402', '1505412', '1508411', ..., 'PAG053550', 'PAG053603', 'PAG113500', 'WQG02231503']

In [148]:
# Confirm that the `npdes_id` hase one unique value per data row, alternate approach.
# Outputs are generally sorted from highest to lowest counts
point_source_df.npdes_id.value_counts()

1503410        1
PA0050016      1
PA0043982      1
PA0044024      1
PA0044423      1
              ..
NJ0169030      1
NJ0170330      1
NJ0171565      1
NJ0171620      1
WQG02231503    1
Name: npdes_id, Length: 812, dtype: int64

In [149]:
# Set index to 'npdes_id'
point_source_df.set_index('npdes_id', inplace=True)

In [150]:
point_source_df.index

CategoricalIndex(['PA0033995', 'PA0051811', '1594403', '1592417', 'NJ0065196',
                  'PA0012238', 'PA0026867', 'PA0030023', 'PA0011185',
                  'NJ0004278',
                  ...
                  'PA0028975', 'NJG0175200', 'NJ0022250', 'NJ0024040',
                  'PA0055671', 'NJ0022985', 'PA0056081', 'PA0036081',
                  'PA0026638', 'NJ0104388'],
                 categories=['1503410', '1505402', '1505412', '1508411', '1586408', '1586409', '1587447', '1590416', ...], ordered=False, dtype='category', name='npdes_id', length=812)

In [151]:
point_source_df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
CategoricalIndex: 812 entries, PA0033995 to NJ0104388
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   ogc_fid       812 non-null    int64   
 1   geom          812 non-null    geometry
 2   city          789 non-null    category
 3   state         812 non-null    category
 4   latitude      812 non-null    float64 
 5   longitude     812 non-null    float64 
 6   huc12         812 non-null    category
 7   avg_n_conc    811 non-null    float64 
 8   lbsn_yr       811 non-null    float64 
 9   mgd           812 non-null    float64 
 10  avgpconc      811 non-null    float64 
 11  lbsp_yr       809 non-null    float64 
 12  kgn_yr        811 non-null    float64 
 13  kgp_yr        809 non-null    float64 
 14  facilityname  812 non-null    category
 15  comid         812 non-null    int64   
dtypes: category(4), float64(9), geometry(1), int64(2)
memory usage: 165.5 

## Save to Parquet Files
The code below converts the data into a locally saved parquet file to avoid having to access the database every time we run the visualization script.  

Apache Parquet has become the high-performance binary cloud format of choice for storing dataframes.
- https://pandas.pydata.org/docs/user_guide/io.html#io-parquet
- https://anaconda.org/TomAugspurger/pandas-performance/notebook
- https://geopandas.readthedocs.io/en/latest/docs/reference/api/geopandas.GeoDataFrame.to_parquet.html

NOTE: The current version of GeoPandas only supports the `pyarrow` engine. In future implementations, look for integration with `fastparquet` engine/library, which is implemented in Pandas and more tightly integrated with numba and dask. See https://fastparquet.readthedocs.io

In [152]:
# Parquet requires pyarrow
import pyarrow

Although Parquet has been tightly integrated with Pandas for a long time and is very stable, the GeoPandas Parquet implementation is relatively new and comes with this warning:

```
# UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a production setting unless you are able to rewrite your Parquet/Feather files.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')
```

In [153]:
import warnings
warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')

In [154]:
# Find current working directory
from pathlib import Path
Path.cwd()

PosixPath('/Users/aaufdenkampe/Documents/Python/WikiSRATMicroService')

In [155]:
# Set relative path - will work for anybody in this directory / cloning the github
data_folder    = Path('data/')

In [156]:
%%time

# Save Model Output Data
# Use the `gzip` compression engine, for bettter (~2/3) but slower (~5x write) compression
# than the default `snappy` engine.

base_df_reach.to_parquet(data_folder /'base_df_reach.parquet',compression='gzip')
rest_df_reach.to_parquet(data_folder /'rest_df_reach.parquet',compression='gzip')

base_df_catch.to_parquet(data_folder /'base_df_catch.parquet',compression='gzip')
rest_df_catch.to_parquet(data_folder /'rest_df_catch.parquet',compression='gzip')

CPU times: user 7.47 s, sys: 136 ms, total: 7.6 s
Wall time: 7.66 s


In [157]:
%%time
# Save Protection Project Data
prot_proj_df.to_parquet(data_folder /'prot_proj_df.parquet',compression='gzip')

CPU times: user 45.2 ms, sys: 3.82 ms, total: 49 ms
Wall time: 51 ms


In [ ]:
%%time
# Save Restoration Project Data
rest_proj_df.to_parquet(data_folder /'rest_proj_df.parquet',compression='gzip')

In [158]:
%%time
# Save Point Source Data
point_source_df.to_parquet(data_folder /'point_source_df.parquet',compression='gzip')

CPU times: user 39.3 ms, sys: 4.06 ms, total: 43.4 ms
Wall time: 45.8 ms
